# Settings

In [ ]:
# Install packages
!pip install optuna
!pip install torcheval
!pip install ipdb
!pip install pydicom
!pip install segment_anything
!pip install scikit-image

# Import classes
from TrainUtils.NetworkTrainer import NetworkTrainer
from DataUtils.XrayDataset import XrayDataset
from Enumerators.NetType import NetType

# Initialize the file system
import s3fs
s3 = s3fs.S3FileSystem()

# Initialize data

In [ ]:
# Define seed
NetworkTrainer.set_seed(111099)

# Define variables
working_dir1 = "s3://dd-s-matteo-dev-resources/"

# Load data
train_data1 = XrayDataset.load_dataset(working_dir=working_dir1, dataset_name="xray_dataset_training", s3=s3)
val_data1 = XrayDataset.load_dataset(working_dir=working_dir1, dataset_name="xray_dataset_validation", s3=s3)
test_data1 = XrayDataset.load_dataset(working_dir=working_dir1, dataset_name="xray_dataset_test", s3=s3)

# Train network

In [ ]:
# Define variables
model_name1 = "resnext50"
net_type1 = NetType.RES_NEXT50
epochs1 = 500
preprocess_inputs1 = True
trial_n1 = None
val_epochs1 = 10
use_cuda1 = True
assess_calibration1 = True
show_test1 = False

# Define trainer
net_params1 = {"n_conv_segment_neurons": 526, "n_conv_view_neurons": 128, "n_conv_segment_layers": 1,
               "n_conv_view_layers": 1, "kernel_size": 3, "n_fc_layers": 2, "optimizer": "Adam",
               "lr_last": 0.005, "lr_second_last_factor": 10, "batch_size": 64}
trainer1 = NetworkTrainer(model_name=model_name1, working_dir=working_dir1, train_data=train_data1,
                          val_data=val_data1, test_data=test_data1, net_type=net_type1, epochs=epochs1,
                          val_epochs=val_epochs1, preprocess_inputs=preprocess_inputs1, net_params=net_params1,
                          use_cuda=use_cuda1, s3=s3)

# Train model
trainer1.train(show_epochs=True)
trainer1.summarize_performance(show_test=show_test1, show_process=True, show_cm=True,
                               assess_calibration=assess_calibration1)

# Evaluate model
trainer1 = NetworkTrainer.load_model(working_dir=working_dir1, model_name=model_name1, trial_n=trial_n1,
                                     use_cuda=use_cuda1, s3=s3)